# **Penting**
- Jangan mengubah atau menambahkan cell text yang sudah disediakan, Anda hanya perlu mengerjakan cell code yang sudah disediakan.
- Pastikan seluruh kriteria memiliki output yang sesuai, karena jika tidak ada output dianggap tidak selesai.
- Misal, Anda menggunakan df = df.dropna() silakan gunakan df.isnull().sum() sebagai tanda sudah berhasil. Silakan sesuaikan seluruh output dengan perintah yang sudah disediakan.
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Pastikan Anda menggunakan variabel df dari awal sampai akhir dan tidak diperbolehkan mengganti nama variabel tersebut.
- Hapus simbol pagar (#) pada kode yang bertipe komentar jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan
- Pastikan Anda mengerjakan sesuai section yang sudah diberikan tanpa mengubah judul atau header yang disediakan.

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [18]:
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
import joblib


def evaluate_model(model, X_test, y_test, average: str = 'weighted'):
    """Hitung metrik utama dan laporkan klasifikasi penuh."""
    y_pred = model.predict(X_test)
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average=average, zero_division=0),
        'recall': recall_score(y_test, y_pred, average=average, zero_division=0),
        'f1_score': f1_score(y_test, y_pred, average=average, zero_division=0),
    }
    report = classification_report(y_test, y_pred)
    return metrics, report


# **2. Memuat Dataset dari Hasil Clustering**
Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [19]:
# Memuat dataset hasil clustering yang memiliki label Target

# Url ini merupakan tautan yang merujuk pada `data_clustering.csv` pada proses clustering sebelumnya
url='https://drive.google.com/uc?id=1xGZLX_ZVXvdqMdNaxWcqGGgsVIIYbkCC'
df = pd.read_csv(url) # Ubah sesuai file tujuan yang diinginkan

print(f'Banyak baris, kolom: {df.shape}')
print('Distribusi label Target:')
print(df['Target'].value_counts().sort_index())


Banyak baris, kolom: (2267, 14)
Distribusi label Target:
Target
0    568
1    557
2    571
3    571
Name: count, dtype: int64


In [20]:
# Menampilkan 5 baris pertama untuk memastikan data siap digunakan
df.head()

,TransactionAmount,TransactionDate,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,TransactionAmount_bin,CustomerAge_bin,Target
0,-0.970546,680,1,36,0,1.419862,0,-0.548393,-0.204629,-0.000315,105,2,2,2
1,0.268963,1178,1,15,0,1.307715,0,0.307960,-0.204629,2.218381,192,3,2,2
2,-0.586526,1262,1,23,2,-1.439874,3,-0.905207,-0.204629,-1.024091,41,1,1,3
3,-0.387294,818,1,33,2,-1.047361,3,-1.347656,-0.204629,0.886694,163,1,1,2
4,-0.972736,1939,0,1,2,0.000000,3,1.121495,-0.204629,0.594263,16,2,0,0


# **3. Data Splitting**
Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [21]:
# Menyiapkan fitur (X) dan label (y) kemudian membagi menjadi data latih dan uji
feature_columns = [col for col in df.columns if col != 'Target']
X = df[feature_columns]
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print('Ukuran data latih:', X_train.shape)
print('Ukuran data uji  :', X_test.shape)


Ukuran data latih: (1813, 13)
Ukuran data uji  : (454, 13)


# **4. Membangun Model Klasifikasi**
Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Menggunakan algoritma klasifikasi yaitu Decision Tree.
2. Latih model menggunakan data yang sudah dipisah.

In [22]:
# Melatih model klasifikasi Decision Tree pada data latih
decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(X_train, y_train)

dt_metrics, dt_report = evaluate_model(decision_tree_model, X_test, y_test)
print('Metrik evaluasi Decision Tree:')
print(pd.DataFrame([dt_metrics]).round(4))
print('Laporan klasifikasi Decision Tree:')
print(dt_report)


Metrik evaluasi Decision Tree:
   accuracy  precision  recall  f1_score
0    0.9978     0.9978  0.9978    0.9978
Laporan klasifikasi Decision Tree:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       114
           1       0.99      1.00      1.00       112
           2       1.00      0.99      1.00       114
           3       1.00      1.00      1.00       114

    accuracy                           1.00       454
   macro avg       1.00      1.00      1.00       454
weighted avg       1.00      1.00      1.00       454



In [23]:
# Menyimpan model Decision Tree terlatih
decision_tree_path = './decision_tree_model.h5'
joblib.dump(decision_tree_model, decision_tree_path)
print(f'Model Decision Tree disimpan ke: {decision_tree_path}')


Model Decision Tree disimpan ke: ./decision_tree_model.h5


# **5. Memenuhi Kriteria Skilled dan Advanced dalam Membangun Model Klasifikasi**



**Biarkan kosong jika tidak menerapkan kriteria skilled atau advanced**

In [24]:
# Melatih model klasifikasi lain untuk eksplorasi (Random Forest)
random_forest_model = RandomForestClassifier(n_estimators=200, random_state=42)
random_forest_model.fit(X_train, y_train)


RandomForestClassifier(n_estimators=200, random_state=42)

In [25]:
# Menampilkan metrik akurasi, presisi, recall, dan F1-score model eksplorasi
rf_metrics, rf_report = evaluate_model(random_forest_model, X_test, y_test)
print('Metrik evaluasi Random Forest:')
print(pd.DataFrame([rf_metrics]).round(4))
print('Laporan klasifikasi Random Forest:')
print(rf_report)


Metrik evaluasi Random Forest:
   accuracy  precision  recall  f1_score
0       1.0        1.0     1.0       1.0
Laporan klasifikasi Random Forest:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       114
           1       1.00      1.00      1.00       112
           2       1.00      1.00      1.00       114
           3       1.00      1.00      1.00       114

    accuracy                           1.00       454
   macro avg       1.00      1.00      1.00       454
weighted avg       1.00      1.00      1.00       454



In [26]:
# Menyimpan model eksplorasi Random Forest
rf_model_path = './explore_random_forest_classification.h5'
joblib.dump(random_forest_model, rf_model_path)
print(f'Model eksplorasi disimpan ke: {rf_model_path}')


Model eksplorasi disimpan ke: ./explore_random_forest_classification.h5


End of Code